In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# URL of the website you are scraping from.
standingsUrl = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"

In [3]:
# 
data = requests.get(standingsUrl)

In [4]:
# Selects the first table which represents the Standings.
soup = BeautifulSoup(data.text)
standingsTable = soup.select('table.stats_table')[0]

# Obtains the links to every team in the Table.
teamLinks = standingsTable.find_all('a')
teamLinks = [link.get("href") for link in teamLinks]
teamLinks = [link for link in teamLinks if "/squads/ in link"]
teamUrls = [f"https://fbref.com{link}" for link in teamLinks]

In [5]:
# 
data = requests.get(teamUrls[0])

In [6]:
# Selects the first table which represents the selected teams' fixtures.
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.8,1.0,65,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.6,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,4.4,0.2,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.8,0.6,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN


In [7]:
# Deletes all the unwanted columns from the table.
del matches["Attendance"]
del matches["Captain"]
del matches["Formation"]
del matches["Referee"]
del matches["Match Report"]
del matches["Notes"]
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.8,1.0,65
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.6,0.1,67
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,4.4,0.2,80
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.8,0.6,61


In [8]:
# Obtains the link to the Shooting Tab of the selected Teams' Page.
soup = BeautifulSoup(data.text)
shootingLink = soup.find_all('a')
shootingLink = [link.get("href") for link in shootingLink]
shootingLink = [link for link in shootingLink if link and "all_comps/shooting/" in link]

shootingLink

['/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions']

In [9]:
data = requests.get(f"https://fbref.com{shootingLink[0]}")

In [10]:
# Selects the Shooting Table of the selected team.
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2021-08-07  17:15  Community Shield  FA Community Shield  Sat   
1          2021-08-15  16:30    Premier League          Matchweek 1  Sun   
2          2021-08-21  15:00    Premier League          Matchweek 2  Sat   
3          2021-08-28  12:30    Premier League          Matchweek 3  Sat   
4          2021-09-11  15:00    Premier League          Matchweek 4  Sat   

                                         ... Standard               Expected  \
     Venue Result GF GA        Opponent  ...     Dist   FK PK PKatt       xG   
0  Neutral      L  0  1  Leicester City  ...      NaN  NaN  0     0      NaN   
1     Away      L  0  1       Tottenham  ...     17.3  1.0  0     0      1.8   
2     Home      W  5  0    Norwich City  ...     18.5  1.0  0     0      2.6   
3     Home      W  5  0         Arsenal  ...     14.8  0.0  0     0      4.4   
4     Away      W  1  0  Leicester City  ...     14.3  0.0  0     0      2.8   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  1.8    0.10 -1.8    -1.8        Match Report  
2  2.6    0.17  1.4     1.4        Match Report  
3  4.4    0.18  0.6     0.6        Match Report  
4  2.8    0.12 -1.8    -1.8        Match Report  

[5 rows x 26 columns]

In [11]:
# Drops the multi-level indexes in the Shooting Table.
shooting.columns = shooting.columns.droplevel()


In [12]:
# Merges the Shooting Table and the Fixtures Table of the selected team.
teamData = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")
teamData.head()


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Sh,SoT,Dist,FK,PK,PKatt
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,12,3,NaN,NaN,0,0
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.8,1.0,65,18,4,17.3,1.0,0,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.6,0.1,67,16,4,18.5,1.0,0,0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,4.4,0.2,80,25,10,14.8,0.0,0,0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.8,0.6,61,25,8,14.3,0.0,0,0


In [13]:
# If you want to scrape data of matches from multiple seasons.
years = list(range(2022,2018,-1))
allMatches = []
years

[2022, 2021, 2020, 2019]

In [14]:
standingsUrl = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"

In [15]:
for year in years:
    data = requests.get(standingsUrl)
    soup = BeautifulSoup(data.text)
    standingsTable = soup.select("table.stats_table")[0]

    teamLinks = [link.get("href") for link in standingsTable.find_all('a')]
    teamLinks = [link for link in teamLinks if "/squads/" in link]
    teamsUrls = [f"https://fbref.com{link}" for link in teamLinks]

    previousSeason = soup.select("a.prev")[0].get("href")
    standingsUrl = f"https://fbref.com{previousSeason}"

    for teamUrl in teamsUrls:
        teamName = teamUrl.split("/")[-1].replace("-Stats", "").replace("-"," ")
        
        data = requests.get(teamUrl)
        matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]
        
        soup = BeautifulSoup(data.text)
        shootingLink = [link.get("href") for link in soup.find_all('a')]
        shootingLink = [link for link in shootingLink if link and "all_comps/shooting/" in link]
        
        data = requests.get(f"https://fbref.com{shootingLink[0]}")
        shooting = pd.read_html(data.text, match = "Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            teamData = matches.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on = "Date")
        except ValueError:
            continue;

        teamData = teamData[teamData["Comp"] == "Premier League"]
        teamData["Season"] = year
        teamData["Team"] = teamName
        del teamData["Attendance"]
        del teamData["Captain"]
        del teamData["Formation"]
        del teamData["Referee"]
        del teamData["Match Report"]
        del teamData["Notes"]
        allMatches.append(teamData)
        time.sleep(5)        



In [16]:
# Obtains the length of the allMatches list
len(allMatches)

80

In [17]:
# Concatenating allMatches to the matchDataFrame and rearranging the columns in the matchDataFrame
matchDataFrame = pd.concat(allMatches)
cols = ["Date","Time","Season","Comp","Round","Day","Venue","Team","Opponent","Result","GF","GA","xG","xGA","Poss","Sh","SoT","Dist","FK","PK","PKatt"]
matchDataFrame = matchDataFrame[cols]
matchDataFrame

,Date,Time,Season,Comp,Round,Day,Venue,Team,Opponent,Result,...,GA,xG,xGA,Poss,Sh,SoT,Dist,FK,PK,PKatt
1,2021-08-15,16:30,2022,Premier League,Matchweek 1,Sun,Away,Manchester City,Tottenham,L,...,1,1.8,1.0,65.0,18.0,4.0,17.3,1.0,0.0,0.0
2,2021-08-21,15:00,2022,Premier League,Matchweek 2,Sat,Home,Manchester City,Norwich City,W,...,0,2.6,0.1,67.0,16.0,4.0,18.5,1.0,0.0,0.0
3,2021-08-28,12:30,2022,Premier League,Matchweek 3,Sat,Home,Manchester City,Arsenal,W,...,0,4.4,0.2,80.0,25.0,10.0,14.8,0.0,0.0,0.0
4,2021-09-11,15:00,2022,Premier League,Matchweek 4,Sat,Away,Manchester City,Leicester City,W,...,0,2.8,0.6,61.0,25.0,8.0,14.3,0.0,0.0,0.0
6,2021-09-18,15:00,2022,Premier League,Matchweek 5,Sat,Home,Manchester City,Southampton,D,...,0,1.0,0.4,64.0,16.0,1.0,16.4,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2019-04-13,12:30,2019,Premier League,Matchweek 34,Sat,Away,Huddersfield Town,Tottenham,L,...,4,0.8,2.5,31.0,7.0,1.0,18.9,1.0,0.0,0.0
36,2019-04-20,15:00,2019,Premier League,Matchweek 35,Sat,Home,Huddersfield Town,Watford,L,...,2,1.0,0.9,46.0,13.0,3.0,18.1,1.0,0.0,0.0
37,2019-04-26,20:00,2019,Premier League,Matchweek 36,Fri,Away,Huddersfield Town,Liverpool,L,...,5,0.2,3.1,31.0,5.0,1.0,21.6,0.0,0.0,0.0
38,2019-05-05,14:00,2019,Premier League,Matchweek 37,Sun,Home,Huddersfield Town,Manchester Utd,D,...,1,1.0,1.8,36.0,7.0,3.0,19.0,1.0,0.0,0.0


In [18]:
# Converts the data frame into a .csv file.
matchDataFrame.to_csv("matches.csv")